In [ ]:
import os
import json

with open("../data/raw/mrbench_v3_devset.json", "r") as f:
    data = json.load(f)

data_structured = data

# Split conversation_history
for i, row in enumerate(data_structured):
    conversation_history_structured = []
    last_flag = ""
    lines = row["conversation_history"].split("\n")
    lines = [line.replace("\u00a0", "").strip() for line in lines if line.strip() != ""]

    # Skip if first line is Student
    if lines and lines[0].startswith("Student:"):
        data_structured[i]["conversation_history_structured"] = []
        continue

    for line in lines:
        if "Tutor:" in line:
            if last_flag == "Tutor":
                conversation_history_structured[-1] = line
            else:
                conversation_history_structured.append(line)
            last_flag = "Tutor"
        elif "Student:" in line:
            if last_flag == "Student":
                conversation_history_structured[-1] = line
            else:
                conversation_history_structured.append(line)
            last_flag = "Student"
        else:
            if conversation_history_structured:
                conversation_history_structured[-1] += " " + line
            else:
                print("Error: No Tutor or Student line found before:", line)

    # Skip if last response is Tutor
    if conversation_history_structured and conversation_history_structured[
        -1
    ].startswith("Tutor:"):
        data_structured[i]["conversation_history_structured"] = []
        continue

    data_structured[i]["conversation_history_structured"] = (
        conversation_history_structured
    )

with open("../data/processed/mrbench_v3_devset.json", "w") as f:
    json.dump(data_structured, f, indent=4)
